# Quantum Generalized Linear Models with Quantopo

Generalized linear models are the simplest instance of link-based statistical
models, which are based on the underlying geometry of an outcome’s underlying
probability distribution (typically from the exponential family).

Machine learning algorithms provide alternative ways to minimize a model’s sum
of square error (error between predicted values and actual values of a test set).

However, some deep results regarding the exponential family’s relation to affine
connections in differential geometry provide a possible alternative to link
functions:
1. Algorithms that either continuously deform the outcome distribution from known results
2. Algorithms that superpose all possible distributions and collapse to fit a dataset 
<br>
-Leveraging the fact that some quantum computer gates, such as the non-Gaussian
transformation gate, essentially perform (1) natively and in a computationally-efficient
way!

This project provides a proof-of-concept for leveraging continuous-variable quantum circuits
and gates to solve the affine connection problem, with benchmarking at state-of-the-art levels
(See powerpoint presentation in quantopo/projects/QGLM/Xanadu/docs/'Quantum Generalized Linear Models.pdf'). 

Xanadu’s qumode formulation makes ideal for implementing quantum GLMs (See powerpoint presentation for greater detail):
1. Ability to perform linear algebra operations on physical data representations.
2. Non-Gaussian transformation gate provides perfect avenue to perform the affine
transformation related to the outcome distribution without a need to specific a
link function to approximate the geometry.

### Packages Imported:

Xanadu's quantum software packages are imported: 

1. strawberryfields (https://strawberryfields.readthedocs.io/en/latest/installing.html) 
2. qmlt (https://qmlt.readthedocs.io/en/latest/installing.html) 

(1) includes the basic gates used in continuous-variable quantum citcuits and (2) is a quantum machine learning package that includes functions for building quantum neural networks and optimization parameters. 

Other packages:

3. Tensorflow
4. numpy
5. pandas

(3) is imported to run program on tensorflow backend-simulator. (4) and (5) are used for statistics and data structure of imported data.

In [ ]:
import strawberryfields as sf
from strawberryfields.ops import Dgate, BSgate, Rgate, Sgate, Kgate, Interferometer
from qmlt.tf.helpers import make_param
from qmlt.tf import CircuitLearner

import tensorflow as tf
import numpy as np
import pandas

### Global variables

depth - depth of neural network (Generalized Linear Models only require 1 layer)<br>
steps - number of training steps when training network.

In [ ]:
depth = 1
steps = 50

### Definition of circuit function

circuit:

The whole function is coded in tensorflow. It takes in a batch of data. The number of qubits is based on the number of features/columns (excluding label). In this example, dimension of features was reduced to 4 using principal component analysis.

The make_param function from the qmlt package helps initialize and keep track of the gate parameters defined in the quantum neural network. See presentation in quantopo/projects/QGLM/Xanadu/docs/'Quantum Generalized Linear Models' to see how the parameters fit in the quantum neural network gates.

The inner function, layer, defines the gates layout per iteration. See presentation in quantopo/projects/QGLM/Xanadu/docs/'Quantum Generalized Linear Models' to get an understanding of this layout.

The circuit backend engine is initiated at the call of sf.Engine() with the number of qumodes (continuous variable qubits) passed to the engine. The with function that follows that is called by eng.run and begins the circuit run for training and testing the neural network.

The parameter optimization is implemented by the call state.quad_expectation(). The output of the circuit is returned.

argument:

X (Tensor) - tensor of input data for training and testing neural network circuit

Return:

Predicted outputs with current circuit setting (Tensor)

In [ ]:
def circuit(X):
    num_qubits = X.get_shape().as_list()[1]

    phi_1 = make_param(name='phi_1', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False)
    theta_1 = make_param(name='theta_1', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False)
    a = make_param(name='a', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False, monitor=True)
    rtheta_1 = make_param(name='rtheta_1', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False, monitor=True)
    r = make_param(name='r', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False, monitor=True)
    kappa = make_param(name='kappa', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False, monitor=True)
    phi_2 = make_param(name='phi_2', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False)
    theta_2 = make_param(name='theta_2', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False)
    rtheta_2 = make_param(name='rtheta_2', stdev=np.sqrt(2)/num_qubits, shape=[depth, num_qubits], regularize=False, monitor=True)

    def layer(i, size):
        Rgate(rtheta_1[i, 0]) | (q[0])
        BSgate(phi_1[i, 0], 0) | (q[0], q[1])
        Rgate(rtheta_1[i, 2]) | (q[1])

        for j in range(size):
            Sgate(r[i, j]) | q[j]

        Rgate(rtheta_2[i, 0]) | (q[0])
        BSgate(phi_2[i, 0], 0) | (q[0], q[1])
        Rgate(rtheta_2[i, 2]) | (q[2])
        BSgate(phi_2[i, 2], theta_2[i, 3]) | (q[2], q[3])
        Rgate(rtheta_2[i, 1]) | (q[1])
        BSgate(phi_2[i, 1], 0) | (q[1], q[2])
        Rgate(rtheta_2[i, 0]) | (q[0])
        BSgate(phi_2[i, 0], 0) | (q[0], q[1])
        Rgate(rtheta_2[i, 0]) | (q[0])
        Rgate(rtheta_2[i, 1]) | (q[1])
        Rgate(rtheta_2[i, 2]) | (q[2])
        Rgate(rtheta_2[i, 3]) | (q[3])
        BSgate(phi_2[i, 2], 0) | (q[2], q[3])
        Rgate(rtheta_2[i, 2]) | (q[2])
        BSgate(phi_2[i, 1], 0) | (q[1], q[2])
        Rgate(rtheta_2[i, 1]) | (q[1])

        for j in range(size):
            Kgate(kappa[i, j]) | q[j]

    eng, q = sf.Engine(num_qubits)

    with eng:
        for i in range(num_qubits):
            Dgate(X[:, i], 0.) | q[i]
        for d in range(depth):
            layer(d, num_qubits)
    
    num_inputs = X.get_shape().as_list()[0]
    state = eng.run('tf', cutoff_dim=10, eval=False, batch_size=num_inputs)
    circuit_output, var0 = state.quad_expectation(0)

    return circuit_output

### Definition of loss function

myloss:

The defined mean-squared loss function. Uses tensorflow's loss function.

Arguments:

circuit_output (Tensor) - circuit output of network to compare with actual output
<br>
targets (Tensor) - the actual output

Return:

The weight mean-squared error float (Tensor)

In [ ]:
def myloss(circuit_output, targets):
    return tf.losses.mean_squared_error(labels=circuit_output, predictions=targets)

### Definition of regularization function

myregularizer:

The defined regularization (if turned on). Uses tensorflow's neural network class for performing L2 regularization. Essentially, it penalises large values of parameters marked for regularization in make_param() under the circuit() funciton above.

Arguments: 

regularized_params (Tensor) - the keyword 'regularized_params' is mandatory; it's used to find the params that have regularize to True in make_param()

Return:

The L2 norm (Tensor)

In [ ]:
def myregularizer(regularized_params):
    return tf.nn.l2_loss(regularized_params)

### Extra: A function for output predictions

outputs_to_predictions:

Used to score a test set and get the predictions for a set of test inputs. First, define how circuit outputs translate to model predictions. The keyword argument circuit_output is mandatory.

Arguments:

circuit_output (Tensor) - the circuit output from output when testing circuit. May also be defined using tensorflow objects

Return:

Circuit output predictions (Tensor)

In [ ]:
def outputs_to_predictions(circuit_output):
    return circuit_output

### Data imported with Pandas and Numpy

Data import for training and testing circuit is done with Pandas data frame, and stored and shuffled with numpy data structure. 

The data set here comes from UCI Forest Fires Dataset (http://archive.ics.uci.edu/ml/datasets/forest%20fires).

In [ ]:
df_train = pandas.read_csv('FFtrain2.csv', header=0)
df_test = pandas.read_csv('FFtest2.csv', header=0)

df_train = np.array(df_train)
df_test = np.array(df_test)

np.random.shuffle(df_train)
np.random.shuffle(df_test)

X_train = df_train[:, 0:4]
Y_train = df_train[:, 4]

X_test = df_test[:, 0:4]
Y_test = df_test[:, 4]

### Hyperparameters

The hyperparameters for the circuit are defined below with a dictionary. 'circuit' is defined by the circuit function above, 'task' specifies the learning task, 'loss' is defined by the loss function, 'optimizer' is specified by the acronym for Stochastic Gradient Descent, 'init_learning_rate' is the learning rate for training, and the 'regularizer' and 'regularization_strength' keys specify the regularization function and strength.

All hyperparameters are passed to CircuitLearner to construct the learner that implements training and optimization with the hyperparameters passed to the CircuitLearner object.

In [ ]:
hyperparams = {'circuit': circuit,
               'task': 'supervised',
               'loss': myloss,
               'optimizer': 'SGD',
                # 'regularizer': myregularizer,
               # 'regularization_strength': 0.1,
               'init_learning_rate': 0.1
               }

learner = CircuitLearner(hyperparams=hyperparams)

### Train

Begin training of circuit, passing in the training data set and specifying the number of training steps and batch size per iteration.

In [ ]:
learner.train_circuit(X=X_train, Y=Y_train, steps=steps, batch_size=10)

### Test

After completion of training, test circuit with test data.

In [ ]:
test_score = learner.score_circuit(X=X_test, Y=Y_test,
                                   outputs_to_predictions=outputs_to_predictions)
print("\nPossible scores to print: {}".format(list(test_score.keys())))
print("Accuracy on test set: ", test_score['accuracy'])
print("Loss on test set: ", test_score['loss'])

Also, run circuit to see output predictions of test input data, after training of circuit.

In [ ]:
outcomes = learner.run_circuit(X=X_test, outputs_to_predictions=outputs_to_predictions)

print("\nPossible outcomes to print: {}".format(list(outcomes.keys())))
print("Predictions for new inputs: {}".format(outcomes['predictions']))
print("Real outputs for new inputs: {}".format(Y_test))

### Conclusion

Results and analysis are presented in our presentation in quantopo/projects/QGLM/Xanadu/docs/'Quantum Generalized Linear Models.pdf'. This is our proof of concept for a programmable quantum generalized linear model using Xanadu's qumode formulation. Our results show that: 

(1) That the qumodes formulation with its unique operators can eliminate the need for link functions within linear models by exploiting the geometry of the models and still give good prediction, 

(2) Better than state-of-the-art prediction for a difficult Tweedie regression dataset (UCI Forest Fire)

(3) Around state-of-the-art prediction for a simulated dataset

(4) This has the potential to bring statistical modeling into quantum computing, by leveraging the underlying geometry and the connection between model geometry and the geometry of quantum physics.

(5) Also a potential avenue through which to implement the homotopy continuation method common in dynamic systems research and some machine learning models (such as homotopy-based LASSO), which take a known problem’s solution and continuously deform it to fit the problem of interest.